Capstone Project - Importing Dataset  

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0           conda-forge
    geopy:          

Reading data from wiki page

In [105]:
import urllib.request
from bs4 import BeautifulSoup
from urllib.request import urlopen

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = urlopen(url)
soup = BeautifulSoup(html,'html.parser')

table = soup.find_all('table')
tables = table[0]

In [107]:
rows = tables.find_all('tr')


In [108]:
PostalCode = []
Borough = []
Neighborhood = []

for row in rows:
    cells = row.find_all('td')
    if len(cells)>1:
        pc = cells[0]
        PostalCode.append (pc.text.strip())
        brgh = cells[1]
        Borough.append(brgh.text.strip())
        ngb = cells[2]
        Neighborhood.append(ngb.text.strip())
            


Creating DataFrame

In [109]:
column_names = ['PostalCode','Borough','Neighborhood']
pd.options.display.max_columns = None
pd.options.display.max_rows = None
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,PostalCode,Borough,Neighborhood


In [110]:
neighborhoods['PostalCode'] = PostalCode
neighborhoods['Borough']= Borough
neighborhoods['Neighborhood'] = Neighborhood

Dropping rows where Borough is not available

In [111]:
neighborhoods['Borough'].value_counts()
neighborhood_data = neighborhoods[neighborhoods['Borough']!= 'Not assigned']


Filling missing Neighborhood data and creating final dataframe

In [113]:
neighborhood_final = neighborhood_data
neighborhood_final.replace(neighborhood_final['Neighborhood']=='Not assigned',n1['Borough'], inplace = True)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [114]:
print (neighborhood_final.shape)

(103, 3)


In [116]:
codes = pd.read_csv('http://cocl.us/Geospatial_data')
codes.rename(columns = {'Postal Code':'PostalCode'}, inplace = True)


In [118]:
ngh_df = pd.merge(neighborhood_final, codes, on ='PostalCode')
ngh_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Getting Toronto Coordinates

In [123]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="tr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Creating Toronto maps using Folium

In [125]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(ngh_df['Latitude'], ngh_df['Longitude'], ngh_df['Borough'], ngh_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto